In [3]:
import stan

/home/hauke/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [2]:
import pystan
model_code = 'parameters {real y;} model {y ~ normal(0,1);}'
model = pystan.StanModel(model_code=model_code)  # this will take a minute
y = model.sampling(n_jobs=1).extract()['y']
y.mean()  # should be close to 0

ModuleNotFoundError: No module named 'pystan'

In [1]:
! pip install pystan fbprophet

  Using cached fbprophet-0.7.1.tar.gz (64 kB)
Using legacy 'setup.py install' for fbprophet, since package 'wheel' is not installed.
    Running setup.py install for fbprophet ... error
    ERROR: Command errored out with exit status 1:
     command: /home/hauke/.pyenv/versions/3.8.6/envs/predict_energy_generation/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-4rv7uxnl/fbprophet_7456fe7176474f9790993f744efab4e7/setup.py'"'"'; __file__='"'"'/tmp/pip-install-4rv7uxnl/fbprophet_7456fe7176474f9790993f744efab4e7/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-t6h1r1y_/install-record.txt --single-version-externally-managed --compile --install-headers /home/hauke/.pyenv/versions/3.8.6/envs/p

In [3]:
import pandas as pd
import fbprophet as Prophet

ModuleNotFoundError: No module named 'fbprophet'